# Tensorflow_Basic

**References**
+ 조대협의 블로그 : http://bcho.tistory.com/1150?category=555440
+ 텐서플로우 API : https://www.tensorflow.org/

텐서플로우의 기본적인 개념과 변수 및 실행방식에 대해 알아보려고 한다. 먼저 아나콘다의 conda install 명령어로 패키지를 설치했다. 아나콘다에서 pip으로 텐서플로우를 깔면 임포트가 제대로 안되는 오류가 발생해서 몇시간 삽질했음

In [2]:
import tensorflow as tf
import numpy as np

## 0. Tensorflow의 작동개념

텐서플로우는 파이썬 기반이지만 라이브러리 함수들로 떡칠이 되어있기 때문에 사용법 및 함수 쓰는 법을 새로 배워야 한다. 얘네는 '텐서'로 그래프, 즉 신경망을 미리 정의하고 '세션'으로 학습을 돌리는 개념을 도입했다. 따라서 단순히 상수들을 더하고 싶을 때에도 아래처럼 그래프를 정의한 뒤 세션을 돌려야 연산이 이루어진다.

In [3]:
# 텐서 정의
a = tf.constant([5], dtype=tf.float32)
b = tf.constant([10], dtype=tf.float32)
c = tf.constant([2], dtype=tf.float32)

# 그래프 정의
d = a*b+c

# 세션 실행
sess = tf.Session()
result = sess.run(d)
print(result)

[52.]


## 1. Constant

Constant는 번역 그대로 **상수형 텐서**이며 다음과 같은 함수원형을 갖는다.

    tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

+ **value** : 상수에 들어갈 값을 넣는다. **단일값**이나 **리스트**를 넣을 수 있으며 추가적으로 shape에서 차원을 지정하면 그 차원만큼의 행렬에 값이 할당된다. 물론 리스트를 사용할 때에는 shape의 차원을 초과해선 안된다.
+ **dtype** : 데이터 타입으로 보통 텐서플로우에서 제공하는 tf.float32형을 사용한다.
+ **shape** : 리스트 형식으로 차원을 입력한다. 사용법은 위에서 다뤘다.
+ **name** : 상수의 이름을 정해주는데 나중에 텐서를 시각화 할 때 표시된다.
+ **verify_shape** : 잘 안씀

## 2. Placeholder

Placeholder는 값들을 담는 **틀 역할을 하는 텐서**이다. 차원과 자료형을 정해주고 **세션단계에서 feed_dict를 통해 값을 먹여주면** 하나씩 차례대로 값을 받아와 모델에 넣어준다. 즉 본인 자체가 연산되는 것이 아니라 값을 받아서 옮겨주는 그릇 역할을 하는 것이다. 같은 차원의 여러 데이터들을 순차적으로 처리할 때 효율적이다. 함수원형은 다음과 같으며 설명은 위에서 한 것과 일치한다. shape를 지정해주지 않아도 알아서 데이터의 차원에 맞춰 처리하는 것 같다.(확인필요)

    tf.placeholder(dtype, shape=None, name=None)

In [10]:
data = [1,2,3,4,5,6]
x = tf.placeholder(dtype=tf.float32, shape=[6])
y = x * 2

sess = tf.Session()
result = sess.run(y, feed_dict={x:data})
print(result)

[ 2.  4.  6.  8. 10. 12.]


## 3. Variable

Variable은 변수로서 우리가 학습을 통해 갱신시켜가는 가중치 등을 정의할 때 사용한다. 위 두 자료형과 다르게 대문자로 시작한다.

    tf.Variable.__init__(initial_value=None, trainable=True, collections=None, validate_shape=True, caching_device=None, name=None, variable_def=None, dtype=None, expected_shape=None, import_scope=None)
    
+ **initial_value** : 텐서 혹은 텐서로 변환가능한 자료형(리스트 등)으로 변수의 초기값을 지정한다. 또한 **세션단계에서 반드시 tf.global_variables_initializer() 함수 등을 사용**해야 그제서야 초기값이 세션에 할당된다.

In [17]:
data2 = [1,2,3,4,5]
x2 = tf.placeholder(tf.float32)
W = tf.Variable([-1], dtype=tf.float32)
y = W*x2

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
result = sess.run(y, feed_dict={x2:data2})
print(result)

[-1. -2. -3. -4. -5.]


## 4. Session

세션은 정의된 그래프를 실제로 돌리는 역할을 한다. 여러가지 사용법이 있는데 대표적인 세 가지 방법을 소개하겠다.

### (1) tf.Session()

가장 기본적인 방법으로 첫 줄은 세션을 정의하고, 다음 줄은 **run()으로 세션을 실행**하고, 마지막 줄은 할당된 자원을 반납하기 위해 세션을 닫는다.

    sess = tf.Session()
    sess.run(...)
    sess.close()
    
    
### (2) with절 사용

가장 많이 보이는 사용법이다. 파이썬의 with절은 해당 절을 벗어나면 자원을 자동 반환하는데 그 점을 이용해 close()를 사용하지 않아도 되는 코드이다.

    with tf.Session() as sess:
        sess.run(...)
        
        
### (3) tf.InteractiveSession()

인터액티브세션은 세션을 정의해 run()으로 돌리지 않더라도 **eval()을 사용해 원하는 그래프를 마음대로 돌릴** 수 있는 편리함이 있다.

    sess = tf.InteractiveSession()
    # 텐서
    a = tf.constant(5)
    b = tf.constant(10)
    
    # 그래프
    c = a*b
    
    # 세션
    c.eval()
    sess.close()
    
  
    
#### 다음 노트북에서는 CNN의 개념 및 기본구성에 대해 공부할 것이고 이후 MNIST, Iris, CIFAR10 등의 데이터로 여러 모델들을 구현해볼 계획이다. (끝)